In [163]:
import numpy as np
import pandas as pd
import requests as req
import time
import re
import glob
import os
import subprocess
from bs4 import BeautifulSoup

In [164]:
### Download Smart Contract Meta Data
if False:
    MAX_PAGE = 1000
    AT_PAGE = 0
    contracts = {"address": [], "block_number": [], "ether": [], "has_source": [] }
    base_url = "https://contract-library.com/api/contracts?"
    for idx in range(MAX_PAGE - AT_PAGE):
        ### Start Downloading
        page_number = idx + 1 + AT_PAGE
        end_point = "{0}n=Ethereum&q=&t=address&s=block_number&o=desc&p={1}&c=100&w=".format(base_url, page_number)
        r = req.get(end_point)
        assert(r.status_code == 200)
        print("=> page_number: {}".format(page_number))
        json = r.json()
        for contract in json["contracts"]:
            contracts["address"].append(contract["address"])
            contracts["block_number"].append(contract["block_number"])
            contracts["ether"].append(contract["ether"])
            contracts["has_source"].append(contract["has_source"])
        ## Write Fragment
        if page_number % 100 == 0:
            csv_file = "assets/addr_{}.csv".format(int(page_number / 100))
            df = pd.DataFrame(contracts)
            df.to_csv(csv_file, header=None, index=False)
            print("=> write to {}".format(csv_file))
            print("=> sleep 2 seconds")
            time.sleep(2)
            ### Clear Contract Data
            contracts["address"] = []
            contracts["block_number"] = []
            contracts["ether"] = []
            contracts["has_source"] = []

In [169]:
## Supported compiler versions
if True:
    versions = []
    for i in range(25 - 18 + 1):
        versions.append("0.4.{}".format(i + 18))
    for i in range(16 - 0 + 1):
        versions.append("0.5.{}".format(i))
    for i in range(3 + 1):
        versions.append("0.6.{}".format(i))

In [170]:
print("=> supported version: {}".format(len(versions)))

=> supported version: 29


In [171]:
### Filter contract source code by version
if False:
    df = pd.read_csv(
        "assets/addr.csv",
        header=None,
        names=["address", "block_number", "ether", "has_source"],
    )
    df_with_source = df[(df.has_source == True)]
    print(df_with_source.describe())
    for address in df_with_source["address"]:
        end_point = "https://etherscan.io/address/{}#code".format(address)
        r = req.get(end_point, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"})
        assert(r.status_code == 200)
        print("=> address: {}".format(address))
        soup = BeautifulSoup(r.text, 'html.parser')
        editor = soup.find(id="editor")
        match = re.findall(r"pragma(\s+)solidity(\s+)[^\d]*((\d+\.?)+)", editor.text)
        print(match)
        if len(match) > 0 and len(match[0]) > 2:
            version = match[0][2]
            if version in versions:
                with open("contracts/{}_{}".format(address, version), "w") as f:
                    f.write(editor.text)

In [172]:
### Report 1
if True:
    num_addr = !cat assets/addr.csv | wc -l
    num_addr = int(num_addr[0])
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    print("=> contracts {}".format(num_addr))
    print("=> sup_contracts {}".format(len(files)))
    print("=> % {}".format(len(files) / num_addr * 100))

=> contracts 100000
=> sup_contracts 1982
=> % 1.982


In [173]:
### Filter smart contract if it can not be compiled 
if False:
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    error_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        !solc use "$SYM_SOLC_VERSION" > /dev/null 2>&1
        output = !solc $SYM_FILE
        if "Error" in ",".join(output):
            print("=> FAIL {}".format(file))
            error_files.append(file)
        else:
            print("=> OK {}".format(file))
    df = pd.DataFrame(error_files)
    csv_file = "contracts/errors.csv"
    df.to_csv(csv_file, header=None, index=False)

In [174]:
### Report 2
if True:
    files = !find contracts/ -not -name "*.csv" -not -name "*.json"
    num_errors = !cat contracts/errors.csv | wc -l
    num_errors = int(num_errors[0])
    print("=> sup_contracts: {}".format(len(files)))
    print("=> error: {}".format(num_errors))
    print("=> comp : {}".format(len(files) - num_errors))
    print("=> % {}".format((len(files) - num_errors)/len(files) * 100))

=> sup_contracts: 1982
=> error: 231
=> comp : 1751
=> % 88.34510595358223


In [175]:
if False:
    files = !find contracts/* -not -name "*.csv" -not -name "*.json"
    df = pd.read_csv("contracts/errors.csv", header=None, names=["contract_path"])
    for contract_path in df["contract_path"]:
        files.remove(contract_path)
    print("=> comp: {}".format(len(files)))
    compiled_files = []
    for idx, file in enumerate(files):
        version = file.split('_')[1]
        os.environ["SYM_SOLC_VERSION"] = version
        os.environ["SYM_FILE"] = file
        os.environ["SYM_FILE_OUTPUT"] = "{}.json".format(file)
        print("=> {} / {} {}".format(idx + 1, len(files), file))
        !solc use $SYM_SOLC_VERSION > /dev/null 2>&1
        !solc --combined-json bin-runtime,srcmap-runtime,ast $SYM_FILE > $SYM_FILE_OUTPUT
        compiled_files.append(file)
    df = pd.DataFrame(compiled_files)
    csv_file = "contracts/compiled.csv"
    df.to_csv(csv_file, header=None, index=False)

In [177]:
### Report 3
if True:
    files = !find contracts/*.json
    num_files = !cat contracts/compiled.csv | wc -l
    print("=> json {}".format(len(files)))
    print("=> num_files {}".format(int(num_files[0])))

=> json 1750
=> num_files 1750


In [162]:
loop_bound = 20

env_content = """
dataload=02
expectCoverage=0.1
maxVisitedBlock={}
maxVisitedBlockBound=50
maxVisitedBlockStep=2
allocatedRange="a0"
""".strip().format(loop_bound)

In [ ]:
### Run symEvm
if True:
    result = {
        "address": [], 
        "contract_name": [],
        "endpoints": [], 
        "covered_jumpis": [], 
        "total_jumpis": [],
        "duration": [],
        "bytelen": [],
        "loop_bound": [],
    }
    df = pd.read_csv("contracts/compiled.csv", header=None, names=["contract_path"])
    coverage_path = "results/coverage.csv"
    with open(".env", "w") as f:
        f.write(env_content)
    if os.path.exists(coverage_path):
         os.remove(coverage_path)
    with open(coverage_path, "w") as f:
        pass
    contract_paths = df["contract_path"]
    for idx, contract_path in enumerate(contract_paths):
        os.environ["SYM_CONTRACT_FILE"] = contract_path
        os.environ["SYM_JSON_FILE"] = "{}.json".format(contract_path)
        print("{} / {}".format(idx + 1, len(contract_paths)))
        ## if json file is empty
        with open(contract_path + ".json", "r") as f:
            if len(f.read()) == 0:
                continue
        ## execute SymEvm
        addr = contract_path.split("/")[1].split("_")[0]
        start_time = time.time()
        output = !node index.js
        duration = time.time() - start_time
        output = '\n'.join(output)
        print("addr: {}".format(addr))
        print(output)
        ## contract
        match = re.findall(r"Start Analyzing Contract:\s+([^\n]+)", output)
        assert(len(match) > 0)
        contract_name = match[0]
        ## endpoints
        match = re.findall(r"endpoints\s+:\s+(\d+)", output)
        assert(len(match) > 0)
        endpoints = int(match[0])
        ## cjumpis
        match = re.findall(r"cjumpis\s+:\s+(\d+)", output)
        assert(len(match) > 0)
        cjumpis = int(match[0])
        ## njumpis
        match = re.findall(r"njumpis\s+:\s+(\d+)", output)
        assert(len(match) > 0)
        njumpis = int(match[0])
        ## byte len
        match = re.findall(r"bytelen\s+:\s+(\d+)", output)
        assert(len(match) > 0)
        bytelen = int(match[0])
        ##
        result["address"].append(addr)
        result["endpoints"].append(endpoints)
        result["covered_jumpis"].append(cjumpis)
        result["total_jumpis"].append(njumpis)
        result["contract_name"].append(contract_name)
        result["duration"].append(round(duration * 1000))
        result["bytelen"].append(bytelen)
        result["loop_bound"].append(loop_bound)
        
    df = pd.DataFrame(result)
    df.to_csv(coverage_path, index=None)

1 / 1750
addr: 0x0063a9c1480d8d667f6232976f3006d6a3edba31
[symEvm] info: Start Analyzing Contract: TokenAuthority
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 11
[symEvm] info: |	cjumpis    : 8
[symEvm] info: |	njumpis    : 8
[symEvm] info: |	bytelen    : 897
[symEvm] info: ----------------------------------------------
2 / 1750
addr: 0x00813626695bd9cbab357eedd45e5083311edeaa
[symEvm] info: Start Analyzing Contract: Zeropool
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 95
[symEvm] info: |	cjumpis    : 97
[symEvm] info: |	njumpis    : 330
[symEvm] info: |	bytelen    : 18575
[symEvm] info: ----------------------------------------------
3 / 1750
addr: 0x00bc3018d474990960145b7dfb10329464434cdc
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpi

addr: 0x02bb94ddc1d378b4bf37c6c46a52848bc818fca9
[symEvm] info: Start Analyzing Contract: Bid
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 116
[symEvm] info: |	cjumpis    : 101
[symEvm] info: |	njumpis    : 106
[symEvm] info: |	bytelen    : 7711
[symEvm] info: ----------------------------------------------
21 / 1750
addr: 0x0308acc5edf725dc6ba65eadf24cbc8353a3b37b
[symEvm] info: Start Analyzing Contract: MCDSaverFlashLoan
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 218
[symEvm] info: |	cjumpis    : 177
[symEvm] info: |	njumpis    : 615
[symEvm] info: |	bytelen    : 36640
[symEvm] info: ----------------------------------------------
22 / 1750
addr: 0x032e5edc2da04c56ed87c1f3ae3db634f1942ec9
[symEvm] info: Start Analyzing Contract: CountdownGriefingEscrow
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 167
[symEvm] info: |	cjumpis    : 151
[symEvm] in

addr: 0x05897681cc33e4fa8fc393588d492e6c6d26d9b7
[symEvm] info: Start Analyzing Contract: MAX
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 158
[symEvm] info: |	cjumpis    : 99
[symEvm] info: |	njumpis    : 99
[symEvm] info: |	bytelen    : 11807
[symEvm] info: ----------------------------------------------
40 / 1750
addr: 0x05c192a8c62e977a1277d8a743efbb3bf19c07c5
[symEvm] info: Start Analyzing Contract: Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 433
[symEvm] info: |	cjumpis    : 66
[symEvm] info: |	njumpis    : 66
[symEvm] info: |	bytelen    : 7410
[symEvm] info: ----------------------------------------------
41 / 1750
addr: 0x05c23ab188a1ed6b0b3ede8235393c528681789e
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEv

addr: 0x0807d0810714d85b49e40349a3002f06e841b7c3
[symEvm] info: Start Analyzing Contract: BasicAdapter
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 133
[symEvm] info: |	cjumpis    : 72
[symEvm] info: |	njumpis    : 81
[symEvm] info: |	bytelen    : 5567
[symEvm] info: ----------------------------------------------
59 / 1750
addr: 0x085ebd3bf879dd8e58781f9c7f2e1fa6b49d2e1f
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13823
[symEvm] info: ----------------------------------------------
60 / 1750
addr: 0x0897381229c6cde4b0d4bfb1d2b15793a24dd762
[symEvm] info: Start Analyzing Contract: TokenMintERC20MintableToken
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 423
[symEvm] info: |	cjumpis    : 71
[symEvm]

addr: 0x0aa8fa3b0b1deb46488250a1755ba58905f99de2
[symEvm] info: Start Analyzing Contract: ProxyAdmin
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 88
[symEvm] info: |	cjumpis    : 47
[symEvm] info: |	njumpis    : 47
[symEvm] info: |	bytelen    : 2947
[symEvm] info: ----------------------------------------------
79 / 1750
addr: 0x0ab81b62603b57865cb2e901145348da1390cba8
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13823
[symEvm] info: ----------------------------------------------
80 / 1750
addr: 0x0aba09a87ab6ba2b4e0db3e01a71e7d55c2f6c49
[symEvm] info: Start Analyzing Contract: AdminUpgradeabilityProxy
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 39
[symEvm] info: |	cjumpis    : 27
[symEvm] info: 

...

addr: 0xe60a89348e1d5262eb934c4bf70cd0e15669af2b
[symEvm] info: Start Analyzing Contract: TokenERC20
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 353
[symEvm] info: |	cjumpis    : 58
[symEvm] info: |	njumpis    : 58
[symEvm] info: |	bytelen    : 5275
[symEvm] info: ----------------------------------------------
1571 / 1750
addr: 0xe63c3c34a59a7fc17137c6cefe85851e7ed525f7
[symEvm] info: Start Analyzing Contract: XPTToken
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 403
[symEvm] info: |	cjumpis    : 53
[symEvm] info: |	njumpis    : 53
[symEvm] info: |	bytelen    : 5135
[symEvm] info: ----------------------------------------------
1572 / 1750
addr: 0xe66d2871720a5bc47e170d8431cff3005535086c
[symEvm] info: Start Analyzing Contract: EthToDaiTradeHelperV4
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 47
[symEvm] info: |	cjumpis    : 31
[symEvm] info: |	n

addr: 0xe804ce3543015332b17fd6e58dc1fcd58b293f3d
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13823
[symEvm] info: ----------------------------------------------
1590 / 1750
addr: 0xe80d347df1209a76dd9d2319d62912ba98c54ddd
[symEvm] info: Start Analyzing Contract: GatewayRegistry
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 367
[symEvm] info: |	cjumpis    : 107
[symEvm] info: |	njumpis    : 123
[symEvm] info: |	bytelen    : 13515
[symEvm] info: ----------------------------------------------
1591 / 1750
addr: 0xe82f10a7b0c6a6cac34289a42107e63d8eef02a2
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info:

addr: 0xe968608915831dea44c0e506309b11ff43626db5
[symEvm] info: Start Analyzing Contract: ContractDeployer
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 477
[symEvm] info: |	cjumpis    : 68
[symEvm] info: |	njumpis    : 68
[symEvm] info: |	bytelen    : 6771
[symEvm] info: ----------------------------------------------
1609 / 1750
addr: 0xe995b519832094fc91699bee316b638911b1c212
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13823
[symEvm] info: ----------------------------------------------
1610 / 1750
addr: 0xe9f787aac3d690b6b437bb3a9959c788ce401f7d
[symEvm] info: Start Analyzing Contract: ProxyAdmin
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 88
[symEvm] info: |	cjumpis    : 47
[symEvm] info: |	n

addr: 0xecfa10c317a1019f9f5219749ba4d305c7db901c
[symEvm] info: Start Analyzing Contract: RaffleItem
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 371
[symEvm] info: |	cjumpis    : 111
[symEvm] info: |	njumpis    : 144
[symEvm] info: |	bytelen    : 13558
[symEvm] info: ----------------------------------------------
1628 / 1750
addr: 0xed1e4edf6c020efe4fc520cfeb4084aebe969111
[symEvm] info: Start Analyzing Contract: PositionTokenSettingsV2
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 78
[symEvm] info: |	cjumpis    : 87
[symEvm] info: |	njumpis    : 108
[symEvm] info: |	bytelen    : 8209
[symEvm] info: ----------------------------------------------
1629 / 1750
addr: 0xed21c5a20070c4a1c8230473bba0144b263efcd0
[symEvm] info: Start Analyzing Contract: DisableLiquidationSpell
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 19
[symEvm] info: |	cjumpis    : 1

addr: 0xef2a6e0ecfee12f38fcf9862474f52d1bec40888
[symEvm] info: Start Analyzing Contract: Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 433
[symEvm] info: |	cjumpis    : 66
[symEvm] info: |	njumpis    : 66
[symEvm] info: |	bytelen    : 7410
[symEvm] info: ----------------------------------------------
1647 / 1750
addr: 0xef59408b7448ce998e08c46b604cac8873a5aa9e
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis    : 150
[symEvm] info: |	bytelen    : 13823
[symEvm] info: ----------------------------------------------
1648 / 1750
addr: 0xef7a985e4ff9b5dccd6eddf58577486887288711
[symEvm] info: Start Analyzing Contract: ERC20Token
[symEvm] info: ----------------------------------------------
[symEvm] info: |	endpoints  : 637
[symEvm] info: |	cjumpis    : 132
[symEvm] info: |	njumpis   